This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [13]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
%pip install selenium chromedriver-binary python-dotenv webdriver-manager



   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/99.6 kB ? eta -:--:--
   ---------------------------------------- 99.6/99.6 kB ? eta 0:00:00


In [5]:
import os
import boto3
from dotenv import load_dotenv

In [6]:
# The file download processing section should be placed here
load_dotenv()
#...........................................................

True

In [7]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'demo-access-key')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'demo-secret-key')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

In [8]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)

In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


options = Options()

# Thiết lập thư mục tải xuống
prefs = {"download.default_directory": r"C:\Users\admin\icttm-test"}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

def find_element(by, locator):
    wait = WebDriverWait(driver, 10)
    return wait.until(EC.presence_of_element_located((by, locator)))

url = "https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing"

driver.get(url)

download_button = find_element(By.XPATH, '//*[@id=":1"]/div/c-wiz/div/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz/div/div/div/div[2]/div/div')
download_button.click()

time.sleep(3)

open_button = find_element(By.XPATH, '/html/body/div[12]/div[4]/div/div[3]/div[2]/div[2]/div[3]')
open_button.click()


In [47]:
from pathlib import Path
import zipfile

zip_file_path = Path('data.zip')
extract_dir = Path('data_raw')

extract_dir.mkdir(parents=True, exist_ok=True)

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

In [48]:
# Upload file to minio
upload_file_to_minio('data_raw/data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data_raw/data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data_raw/data3.json', 'data', 'data-raw/data3.json')